# S3 Example Notebook

This notebook demonstrates how to interact with the `world-modelers` S3 bucket for data and model storage.

### Configuration

First, ensure that the appropriate credentials are stored in your AWS credentials at `~/.aws/credentials`. These should be stored under the wmuser profile with something like:

```
[wmuser]
aws_access_key_id = WMUSER_ACCESS_KEY
aws_secret_access_key = WMUSER_SECRET_KEY
```

> Note that this profile must be specified by name when creating the boto3 session.


### Requirements
This notebook was tested with the following versions of `boto3` and `requests` but should be compatible with most other recent versions of these libraries:

```
pip install boto3==1.9.120
pip install requests==2.21.0
```

In [1]:
import boto3
import requests

profile = "wmuser"
bucket_name = "world-modelers"

First we can instantiate our connection with S3:

In [2]:
session = boto3.Session(profile_name=profile)

s3 = session.resource("s3")
s3_client = boto3.client("s3")

Now we can inspect the bucket and list the keys it contains:

In [3]:
bucket = s3.Bucket(bucket_name)

In [4]:
for obj in bucket.objects.all():
    print(obj.key)

applications/
applications/BioNetGen-2.3.1-Linux.tar.gz
applications/stanford-corenlp-full-2018-10-05.zip
data/
data/DySE_tests.ipynb
data/NationalFile_20181001.zip
data/WordNet-3.0-glosstag.tar.bz2
data/WordNet-3.0.tar.bz2
data/delphi_data.zip
data/glove.840B.300d.txt.tgz
data/historical_nbg_maiz.json
data/news_articles.zip
data/style.js
data/vectors.txt
example_s3_file.txt
indra_models/test/curations.json
indra_models/test/raw_statements.json
indra_models/test/statements.json
repos/
repos/dyse-wm-example.zip
repos/learnit_release_v1.tar.gz
repos/lemma.nv
results/fsc_model/1559175778.zip
results/fsc_model/1559177478.zip
results/fsc_model/cf49488892ea86c77f0f0a4176e73cfa112b52db9126d9236db9f4d3e0e21a17.zip
results/fsc_model/cf999d9927c4efca36a4285be2ae06266570b6bd997953249567e97fbcab2d2f.zip
results/fsc_model/f9c796787d534ba92ff1528353e045921a74b56ef9c20cd54282d0073d2015c3.zip
results/malnutrition_model/maln_raster_hires_baseline.csv
results/malnutrition_model/test.csv


Next, we can generate a test file called `example.txt`.

In [5]:
file_name = "example.txt"
s3_key = "example_s3_file.txt"

with open(file_name, "w") as f:
    f.write("Testing 123...")

We can upload this to S3. 

> Note that the `ExtraArgs` provided here are optional. These `ExtraArgs` make the uploaded file public. This means that the file will be available through an HTTP request.

In [6]:
s3_client.upload_file(file_name, 
                      bucket_name, 
                      s3_key,  # here we specify the key we wish the file to have in S3
                      ExtraArgs={'ACL':'public-read'}) # here we make the file public

We can download this file using `boto3` if we wish:

In [7]:
bucket.download_file(s3_key, file_name)


We can also execute an HTTP request to obtain the file since we have made it public. This should be made to `https://world-modelers.s3.amazonaws.com/OBJECT_KEY` where you replace `OBJECT_KEY` with the key we specify above.

In [8]:
url = "https://world-modelers.s3.amazonaws.com/{}".format(s3_key)
requests.get(url).text

'Testing 123...'